In [37]:
import figaro_retriever
import keras
import numpy as np
import pandas as pd
import nltk
import random
import tensorflow
sess = tensorflow.Session(config = tensorflow.ConfigProto(log_device_placement=True))

import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.optimizers import Adam


import sys

In [67]:
retriever = figaro_retriever.LyricRetriever("Kenny Chesney")

In [68]:
raw = retriever.lyrics.lower().split("\n")
raw = [x for x in raw if x]
raw

[' ',
 'we gather tonight for a celebration',
 'under the flag of a no-shoes nation',
 'speakin our gospel, seekin\u2005our\u2005truth',
 'brothers in arms\u2005and sisters and cowboys',
 '(who gets to\u2005live like we do?)',
 'raisin the roof',
 '(who gets to live like\u205fwe\u205fdo?)',
 'we\u205fcome down from\u205fthe mountains, come\u205fup from the coast',
 'were all comin out of some kind of comatose',
 'fill up our cups with a little bit of freedom',
 'come on, hold em up, everybody let me see em',
 'yeah, we laugh and we love and we cant get enough',
 'as loud as were livin and we still turn it up',
 'were amplified, come alive, side by side',
 'no matter where it goes, you know were here for the ride',
 'who gets to roll like we do? (we do)',
 'yeah, who wants to rock like we do? (we do)',
 'yeah, who aint gonna stop, man, no matter what?',
 'and who gets to live like we do? (we do)',
 'we drag this circus from town to town',
 'full of jesters, and jokers, and barroom clown

Using a tokenizer to represent words with numbers

In [69]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(raw)
total_words = len(tokenizer.word_index) + 1

In [70]:
input_sequences = []

for line in raw:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

In [71]:
max_sequence_len = max([len(x) for x in input_sequences])

In [72]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = "pre"))

In [73]:
x_train = input_sequences[:,:-1]
y_train = input_sequences[:,-1]

In [74]:
y_train = keras.utils.to_categorical(y_train, num_classes = total_words)

In [75]:
model = Sequential()
model.add(Embedding(total_words, 240, input_length = max_sequence_len - 1))
model.add(Bidirectional(LSTM(75, return_sequences = True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(75)))
model.add(Dense(total_words, activation = "softmax"))
adam = Adam(lr = .001)

In [76]:
model.compile(loss = "categorical_crossentropy", optimizer = adam, metrics = ["accuracy"])

In [77]:
history = model.fit(x_train, y_train, epochs = 100, verbose = 1)

Epoch 1/100
22168/22168 [==============================] - 73s 3ms/sample - loss: 6.2286 - acc: 0.0484
Epoch 2/100
22168/22168 [==============================] - 74s 3ms/sample - loss: 5.7096 - acc: 0.0638
Epoch 3/100
22168/22168 [==============================] - 68s 3ms/sample - loss: 5.4027 - acc: 0.0772
Epoch 4/100
22168/22168 [==============================] - 68s 3ms/sample - loss: 5.1047 - acc: 0.0974
Epoch 5/100
22168/22168 [==============================] - 67s 3ms/sample - loss: 4.8173 - acc: 0.1255
Epoch 6/100
22168/22168 [==============================] - 67s 3ms/sample - loss: 4.5474 - acc: 0.1583
Epoch 7/100
22168/22168 [==============================] - 68s 3ms/sample - loss: 4.2940 - acc: 0.1904
Epoch 8/100
22168/22168 [==============================] - 67s 3ms/sample - loss: 4.0622 - acc: 0.2228
Epoch 9/100
22168/22168 [==============================] - 67s 3ms/sample - loss: 3.8441 - acc: 0.2510
Epoch 10/100
22168/22168 [==============================] - 68s 3ms/sampl

22168/22168 [==============================] - 67s 3ms/sample - loss: 0.5498 - acc: 0.8517
Epoch 80/100
22168/22168 [==============================] - 67s 3ms/sample - loss: 0.5462 - acc: 0.8521
Epoch 81/100
22168/22168 [==============================] - 67s 3ms/sample - loss: 0.5356 - acc: 0.8541
Epoch 82/100
22168/22168 [==============================] - 67s 3ms/sample - loss: 0.5365 - acc: 0.8526
Epoch 83/100
22168/22168 [==============================] - 67s 3ms/sample - loss: 0.5400 - acc: 0.85102s - loss:
Epoch 84/100
22168/22168 [==============================] - 68s 3ms/sample - loss: 0.5364 - acc: 0.8539
Epoch 85/100
22168/22168 [==============================] - 68s 3ms/sample - loss: 0.5357 - acc: 0.8519
Epoch 86/100
22168/22168 [==============================] - 67s 3ms/sample - loss: 0.5207 - acc: 0.8576
Epoch 87/100
22168/22168 [==============================] - 68s 3ms/sample - loss: 0.5201 - acc: 0.8556
Epoch 88/100
22168/22168 [==============================] - 67s 3ms

In [78]:
def generate_lyrics(next_words = 10):
    #seed_text = tokenizer.sequences_to_texts([[np.random.randint(0, len(tokenizer.word_index))]])
    seed_text = random.choice(list(tokenizer.word_index.keys()))


    for x in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen = max_sequence_len - 1, padding = "pre")
        predicted = model.predict_classes(token_list, verbose = 0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [83]:
song_lyrics = ""

for x in range(20):
    #song_lyrics += generate_lyrics() + " \n "
    print(generate_lyrics(next_words = 5))
    if x % 4 == 0 and x != 0:
        print()

bottles breakin jukebox buzzin 30 up
devils face the devil in her
roulette park truck stop faded little
toker i just kept shining down
add out on the breeze the

centuries he could be another one
ever burnt ready to know what
clippings little people just waiting on
moments with her now theyre all

grind to the car are that
sure were livin but we were
york to soak up else in
among the sun and the sand

pirates just gotta let it take
imagination time i hear that song
win he could be another one
tamed he could be another one

bottom and i see this bar
rocks he could be another one
point to stash dont you dont


In [82]:
model.save("Kenny Chesney 86")